In [1]:
!pip install apache-beam[gcp]


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 89.7/89.7 kB 2.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.4/50.4 kB 687.1 kB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 152.0/152.0 kB 6.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.5/43.5 kB 2.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 173.5/173.5 kB 11.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 38.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 193.8/193.8 kB 11.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 304.6/304.6 kB 17.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 184.7/184.7 kB 10.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━

In [2]:
import apache_beam as beam
from apache_beam.options.pipeline_options import PipelineOptions


Use Case: Sensor Data Analysis

I will simulate the processing of temperature readings from multiple sensors in a smart home environment. The readings are coming in real-time, and we need to process this data using Apache Beam with features like composite transforms, pipeline I/O, triggers, windowing, ParDo, and streaming.



1. Composite Transform

We will create a composite transform to process sensor readings by filtering and scaling temperatures.

In [3]:
class ScaleAndFilter(beam.PTransform):
    def expand(self, pcoll):
        return (
            pcoll
            | 'Scale Temperature' >> beam.Map(lambda x: (x[0], x[1] * 1.8 + 32))  # Convert °C to °F
            | 'Filter Normal' >> beam.Filter(lambda x: 60 <= x[1] <= 80)  # Keep only normal temperatures
        )


2. Pipeline I/O

Here, we'll simulate reading sensor data from a file and writing the output to another file.

In [4]:
def io_pipeline():
    with beam.Pipeline(options=PipelineOptions()) as pipeline:
        (
            pipeline
            | 'Read from file' >> beam.io.ReadFromText('sensor_data.txt')  # Simulate reading from a file
            | 'Parse sensor data' >> beam.Map(lambda x: (x.split(',')[0], float(x.split(',')[1])))
            | 'Scale and Filter' >> ScaleAndFilter()
            | 'Write to file' >> beam.io.WriteToText('output_sensor_data.txt')
        )


3. Triggers and Windowing

We'll use windowing and triggers to analyze data in fixed time windows and trigger outputs based on processing time.

In [5]:
def windowing_pipeline():
    with beam.Pipeline(options=PipelineOptions()) as pipeline:
        (
            pipeline
            | 'Create sensor data' >> beam.Create([('sensor1', 65), ('sensor2', 75), ('sensor1', 85)])
            | 'Window into fixed intervals' >> beam.WindowInto(
                beam.window.FixedWindows(10),  # 10-second windows
                trigger=beam.trigger.AfterProcessingTime(10),  # Trigger after 10 seconds
                accumulation_mode=beam.trigger.AccumulationMode.DISCARDING
            )
            | 'Group by sensor' >> beam.GroupByKey()
            | 'Print grouped data' >> beam.Map(print)
        )


4. ParDo

We'll use ParDo to perform a more complex transformation, like flagging abnormal temperatures.

In [6]:
class FlagAbnormalTemps(beam.DoFn):
    def process(self, element):
        sensor, temp = element
        if temp < 60 or temp > 80:
            yield f'Warning: {sensor} reported abnormal temperature {temp}'

def pardo_pipeline():
    with beam.Pipeline(options=PipelineOptions()) as pipeline:
        (
            pipeline
            | 'Create sensor data' >> beam.Create([('sensor1', 50), ('sensor2', 85), ('sensor3', 75)])
            | 'Flag abnormal temps' >> beam.ParDo(FlagAbnormalTemps())
            | 'Print warnings' >> beam.Map(print)
        )


5. Streaming

Simulating streaming data from sensors (in a real-world scenario, we would use Pub/Sub, etc.).

In [7]:
import time

def streaming_pipeline():
    with beam.Pipeline(options=PipelineOptions(streaming=True)) as pipeline:
        (
            pipeline
            | 'Simulate sensor data' >> beam.Create([('sensor1', 65), ('sensor2', 85), ('sensor3', 45)])
            | 'Simulate streaming' >> beam.ParDo(lambda x: (time.sleep(1), print(x)))
        )

streaming_pipeline()


('sensor1', 65)
('sensor2', 85)
('sensor3', 45)


Summary of Approach:

Composite Transform: Scale and filter sensor data.

Pipeline I/O: Read sensor data from a file and write the output.

Triggers and Windowing: Analyze data in fixed intervals with triggers.

ParDo: Detect and flag abnormal temperatures.

Streaming: Simulate real-time sensor data processing.